# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando o classificador [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Nesse modelo vamos passar a considerar os textos das íntegras.

A primeira passagem, serão criadas colunas com as palavras-chaves informadas pela área negocial que vão indicar a ausência ou presença do termo no texto da íntegra da peça de promoção do arquivamento.

Na segunda passagem, vamos passar a considerar unicamente o texto da íntegra.

**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados.

Vamos ler os textos das íntegras, limpá-los e associar ao dataframe com os dados de homologação.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from sklearn import metrics

In [3]:
PATH = "../data/"

In [4]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/andre/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/andre/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# carga dos textos

from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('portuguese')
stop_words.extend(['n', 'nº', 'n°', 'n.º', ',', '.', '!', '?', ';', ':', '...', 'º', '–', '/', '(', ')'])

def get_text(file):
    with open(file, encoding='utf-8', errors='replace') as f:
        data=' '.join(line.strip() for line in f)
        f.close()
    
    return data.strip()

def get_text_non_stop_words(text):
    filtered_text = ''
    for w in word_tokenize(text, language='portuguese'):
        if w not in stop_words:
            filtered_text += w
            filtered_text += ' '            
    return filtered_text.strip()   
    

folder_integras = f'{PATH}/integras-textos'

texts = {}
for file in listdir(folder_integras):
    if isfile(join(folder_integras, file)):
        texts[file.split('.')[0]] = get_text(join(folder_integras,file))
    else:
        print('is not file', file)

    
#texts['71078312']

In [8]:
# montando um DataFrame
df_original = pd.read_json(f'{PATH}/1A.CAM.homologacao-arquivamento.json')
df_work = df_original.copy()
#df_work.dropna(subset=['identificadorPecaPromocao'], inplace=True)
#df_work.reset_index(drop=True, inplace=True)
cols = ['id', 'homologado']
df_work.drop(inplace=True, columns=[col for col in df_work.columns if col not in cols])
df_work['peca_promocao'] = None
df_work.sample(5)

,homologado,id,peca_promocao
1111,1,72981233,None
5765,1,66708648,None
5506,1,66930820,None
3466,1,67103920,None
3790,1,68038311,None


In [9]:
# Associando o texto...
for key, text in zip(texts.keys(), texts.values()):
    if key != '' and text != '':
        df_work.loc[df_work.id == int(key), 'peca_promocao'] = text
        
df_work.sample(5)

,homologado,id,peca_promocao
5132,1,69210378,EXMO MINISTÉRIO PÚBLICO FEDERAL Procuradoria...
2315,1,75810109,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...
3295,1,74118214,EXMO MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA...
5597,1,67897371,MINISTÉRIO PÚBLICO FEDERAL Procuradoria da Rep...
1957,1,76005649,PR-MS-00025324/2017 Ministério Público Federa...


In [10]:
print(len(df_work[df_work['peca_promocao'].isnull()]))
df_work.dropna(subset=['peca_promocao'], inplace=True)
df_work.reset_index(drop=True, inplace=True)
print(len(df_work[df_work['peca_promocao'].isnull()]))

1401
0


In [11]:
print(len(df_work[df_work['homologado'] == 1]), len(df_work[df_work['homologado'] == 0]))

4742 73


In [12]:
#Carga das palavras chaves
palavras_chaves = pd.read_csv(f'{PATH}/1A.CAM.palavras-chaves.txt')
palavras_chaves

,palavras-chaves,expressao-regular
0,AUSÊNCIA DE IRREGULARIDADES,\bausência[s]?\b(\s)+\bde\b(\s)+\birregularid...
1,IRREGULARIDADE SANADA,\birregularidade[s]?\b(\s)+\bsanada[s]?\b
2,NÃO COMPROVAÇÃO,\bnão\b(\s)+\bcomprovação\b
3,INEXISTÊNCIA,\binexistência[s]?\b
4,SOLUCIONADA,\bsolucionada[s]?\b
5,EXAURIMENTO,\bexaurimento\b
6,EXAURIDA,\bexaurida[s]?\b
7,PERDA DE OBJETO,\bperda\b(\s)+\bde\b(\s)+\bobjeto\b
8,IRREGULARIDADE NÃO COMPROVADA,\birregularidade[s]?\b(\s)+\bnão\b(\s)+\bcomp...
9,INEXISTÊNCIA DE INDÍCIOS DE IRREGULARIDADE,\binexistência\b(\s)+\bde\b(\s)+\bindício[s]?...


## Busca de palavras-chaves nos textos das peças de promoção

Dada a lista de palavras-chaves anteriormente listada, vamos procurá-la nos textos das íntegras.

Vamos criar uma coluna para cada uma das palavras-chaves e marcar com ausência ou presença da mesma no texto correspondente.

Vamos utilzar expressões regulares.

In [13]:
pcs = palavras_chaves['palavras-chaves']

# vou ignorar as colunas com o caractere '+' // PRECISAMOS TRATAR ISSO AINDA (e ainda temos o problema dos espaços)
for pc in pcs:
    if pc.rfind('+') == -1:
        df_work[pc] = 0

df_work.sample(5)

,homologado,id,peca_promocao,AUSÊNCIA DE IRREGULARIDADES,IRREGULARIDADE SANADA,NÃO COMPROVAÇÃO,INEXISTÊNCIA,SOLUCIONADA,EXAURIMENTO,EXAURIDA,...,RECOMENDAÇÃO EXPEDIDA E ACATADA,JUSTIFICATIVAS SATISFATÓRIAS,CORREÇÃO DAS FALHAS,AÇÃO AJUIZADA,QUESTÃO JUDICIALIZADA,DIREITO INDIVIDUAL,EXCESSO DE PESO,MAMOGRAFIA,SISCAN,MAMÓGRAFO (ADICIONADA)
1609,1,56820896,PRM-LAJ-RS-00000232/2018 Referência: IC1.29.0...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2171,1,62349388,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3954,1,62541570,Procedimento Preparatório nº.: 1.33.009.000001...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
758,1,73983456,Ref.: Inquérito Civil nº 1.15.000.002477/2016-...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2594,1,48276613,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
import re


peca = df_work['peca_promocao'][4062]
result = re.search(r'\bdireito\b(\s)+\bindividua(l|is)\b', peca, re.IGNORECASE)
result

<_sre.SRE_Match object; span=(2798, 2816), match='direito individual'>

In [15]:
#import re

#for v in palavras_chaves['palavras-chaves']:
#    for i in range(len(df_work)):
#        #v_with_spaces = re.sub(' ', '\s+', v)
#        if re.search(r"\b" + v + r"?\b", df_work['peca_promocao'][i], re.IGNORECASE):
#            df_work.loc[i, v] = 1
            
#df_work.sample(10)

# Correlacionando as palavras-chaves com a homologação/não homologação

A ideia aqui é mostrar o percentual de presença/ausência de palavras chaves e suas relações com a variável dependente.

# Treinando um modelo com Random Forest

In [28]:
# vamos treinar o modelo
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df_work, random_state=42)
train.shape, valid.shape

((3611, 28), (1204, 28))

In [35]:
cols_to_remove = ['id', 'peca_promocao', 'homologado']
features = [col for col in df_work.columns if col not in cols_to_remove]

model = RandomForestClassifier(random_state=42)
model.fit(train[features], train['homologado'])


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [36]:
model.score(df_work[features], df_work['homologado'])

0.9846313603322949

In [39]:
train_preds = model.predict(train[features])
valid_preds = model.predict(valid[features])

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def print_score():
    print('Accuracy score:', accuracy_score(valid['homologado'], valid_preds))
    print('Precision score: ', format(precision_score(valid['homologado'], valid_preds)))
    print('Recall score: ', format(recall_score(valid['homologado'], valid_preds)))
    print('F1 score: ', format(f1_score(valid['homologado'], valid_preds)))

    
print_score()

Accuracy score: 0.9842192691029901
Precision score:  0.9850374064837906
Recall score:  0.9991568296795953
F1 score:  0.9920468815403936


In [40]:
from sklearn.metrics import confusion_matrix

confusion_matrix(valid['homologado'], valid_preds).ravel()

array([   0,   18,    1, 1185], dtype=int64)